## Titanic

### pre requierment

#conda install catboost

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# EDA Automation
from pandas_profiling import ProfileReport

#Preprocess, split and metrics
from sklearn.preprocessing import LabelEncoder, OneHotEncoder , StandardScaler , Normalizer
from sklearn.model_selection import cross_validate, train_test_split , StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score ,roc_auc_score ,roc_curve

#Modeling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier, AdaBoostClassifier
#XGBoost
from xgboost import XGBClassifier
#CATBoost
from catboost import CatBoostClassifier
#LightGBM
from lightgbm  import LGBMClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
SEED = 123
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


#### importing data

In [2]:
df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.shape , df_test.shape

((891, 12), (418, 11))

In [5]:
df = pd.concat([df_train, df_test], axis=0)
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [6]:
df.shape

(1309, 12)

### descriptive data

In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


### Droping unnecessery columns

In [9]:
df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis= 1, inplace = True)

In [10]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,3,male,22.0,1,0,7.2500,S
1,1.0,1,female,38.0,1,0,71.2833,C
2,1.0,3,female,26.0,0,0,7.9250,S
3,1.0,1,female,35.0,1,0,53.1000,S
4,0.0,3,male,35.0,0,0,8.0500,S


## features that need to be numeric

In [11]:
df.select_dtypes("O")

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S
...,...,...
413,male,S
414,female,C
415,male,S
416,male,S


In [12]:
df.groupby("Sex").size()

Sex
female    466
male      843
dtype: int64

In [13]:
df.groupby(["Embarked"]).size()

Embarked
C    270
Q    123
S    914
dtype: int64

In [14]:
le = LabelEncoder()

In [15]:
df["Sex"]= le.fit_transform(df["Sex"])
df["Embarked"]= le.fit_transform(df["Embarked"])

In [16]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,3,1,22.0,1,0,7.2500,2
1,1.0,1,0,38.0,1,0,71.2833,0


### find and fill null value

In [17]:
df.isnull().sum()

Survived    418
Pclass        0
Sex           0
Age         263
SibSp         0
Parch         0
Fare          1
Embarked      0
dtype: int64

In [18]:
[column for column in df.columns if df[column].isnull().sum()>0]

['Survived', 'Age', 'Fare']

### Survived is target feature and so it doesn't need fill, lets do it for Age and Fare

In [19]:
df["Age"]= df["Age"].fillna(df["Age"].median())
df["Fare"]= df["Fare"].fillna(df["Fare"].mode()[0])

In [20]:
df.isna().sum()

Survived    418
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

### Split data to train and test

In [21]:
df_test = df[df["Survived"].isna()]
df_train = df[df["Survived"].isna()==False]

In [22]:
X_df_test =df_test.drop("Survived", axis=1)

In [23]:
norm = Normalizer()
X_df_test[["Age", "Fare"]] =norm.fit_transform(X_df_test[["Age", "Fare"]])
X_df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,0.975204,0,0,0.221306,1
1,3,0,0.989090,1,0,0.147311,2
2,2,1,0.988012,0,0,0.154377,1
3,3,1,0.952194,0,0,0.305495,2
4,3,0,0.873055,1,1,0.487621,2
...,...,...,...,...,...,...,...
413,3,1,0.961069,0,0,0.276307,2
414,1,0,0.337158,0,0,0.941448,0
415,3,1,0.982727,0,0,0.185059,2
416,3,1,0.961069,0,0,0.276307,2


## To build a model, it is necessary to separate the target variable(y) from the predictor variables(X)

In [24]:
y = df_train["Survived"]
X = df_train.drop(["Survived"], axis=1)

In [25]:
X.shape , y.shape

((891, 7), (891,))

In [26]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2


There are two features that need to scaling

In [27]:
norm = Normalizer()

In [28]:
X[["Age", "Fare"]] =norm.fit_transform(X[["Age", "Fare"]])
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,0.949757,1,0,0.312988,2
1,1,0,0.470417,1,0,0.882444,0
2,3,0,0.956551,0,0,0.291564,2
3,1,0,0.550338,1,0,0.834942,2
4,3,1,0.974555,0,0,0.224148,2


# LGBM or Light Gradian Boosting Model

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.3 , random_state=SEED)
kf = StratifiedKFold(n_splits=10 ,shuffle=True)
lgb = LGBMClassifier(random_state=SEED)
scores = cross_validate(lgb, X, y, scoring=('roc_auc',"accuracy","f1"),cv=kf, n_jobs=-1, return_estimator=True)
model_lgb = scores["estimator"]

In [30]:
y_hats = [model.predict(X_test) for model in model_lgb]
accuracy = np.mean([accuracy_score(y_hat, y_test) for y_hat in y_hats])
print(f'LGBM Accuracy(train data) = {accuracy:.4f}')

LGBM Accuracy(train data) = 0.9425


# XGB or eXtreme Gradient Boosting Model

In [31]:
xgb = XGBClassifier(booster = "gbtree" , learning_rate = 0.3 , n_estimators = 10, random_state=SEED)
scores = cross_validate(xgb,X_train, y_train,scoring="accuracy",cv=kf, n_jobs=-1, return_estimator=True)
model_xgb = scores["estimator"]

In [32]:
y_hats= [model.predict(X_test) for model in model_xgb]
acc = np.mean([accuracy_score(y_test, y_hat) for y_hat in y_hats])
print(f'XGB Accuracy(train data) = {acc:.4f}')

XGB Accuracy(train data) = 0.8190


#### The best parametrs for XGB, we use Grid Search

In [33]:
from sklearn.model_selection import GridSearchCV

In [34]:
parameters = {
    'booster': ['gbtree', 'gblinear', 'dart'],
    'learning_rate': [0.1, 0.3, 0.6, 0.9, 1],
    'n_estimators': [0.1,1,10, 15]
}
grid = GridSearchCV(xgb, param_grid=parameters,cv=5 , verbose=0 )
grid.fit(X_train, y_train)
print(grid.best_params_)

{'booster': 'gbtree', 'learning_rate': 0.3, 'n_estimators': 10}


# RFC or RandomForestClassifier model

In [35]:
rfc = RandomForestClassifier(criterion='entropy' ,max_features="auto", n_estimators=15 ,random_state=SEED)
scores = cross_validate(rfc , X_train, y_train, scoring="accuracy",cv=kf, n_jobs=-1, return_estimator=True)
model_rfc = scores["estimator"]

In [36]:
y_hats = [model.predict(X_test) for model in model_rfc]
acc = np.mean([accuracy_score(y_test, y_hat) for y_hat in y_hats])
print(f'RFC Accuracy(train data) = {acc:.4f}')

RFC Accuracy(train data) = 0.7937


### The best parametrs for RFC

In [37]:
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier()
parameters = {
    'criterion': [ "gini" , "entropy"],
    'max_features' : ["auto", "sqrt", "log2"],
    'n_estimators': [15, 50, 100, 200]
}
grid = GridSearchCV(rfc, param_grid=parameters,cv=5 , verbose=1 )
grid.fit(X_train, y_train)
print(grid.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
{'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 15}


## GBC or GradientBoostingClassifier Model

In [38]:
gbc = GradientBoostingClassifier(criterion= 'squared_error',
                                learning_rate= 0.3,
                                loss= 'deviance',
                                n_estimators= 10 ,
                                random_state=SEED)
scores = cross_validate(gbc, X_train, y_train, scoring="accuracy",cv=kf, n_jobs=-1, return_estimator=True)
model_gbc = scores["estimator"]

In [39]:
y_hats = [model.predict(X_test) for model in model_gbc]
acc = np.mean([accuracy_score(y_test, y_hat) for y_hat in y_hats])
print(f'GBC Accuracy(train data) = {acc:.4f}')

GBC Accuracy(train data) = 0.8228


### The best parametrs for GBC

In [40]:
gb = GradientBoostingClassifier()
parameters = {
    'loss': [ 'deviance', 'exponential'],
    'criterion': ['friedman_mse', 'squared_error'],
    'learning_rate' : [0.1, 0.3 , 10],
    'n_estimators': [10, 100, 200]
}
grid = GridSearchCV(gb, param_grid=parameters,cv=5 , verbose=1 )
grid.fit(X_train, y_train)
print(grid.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'deviance', 'n_estimators': 10}


## CAT or CatBoost Model

In [41]:
cat = CatBoostClassifier()
scores = cross_validate(cat , X_train, y_train, scoring="accuracy",cv=kf, n_jobs=-1, return_estimator=True)
model_cat = scores["estimator"]
clear_output()

In [42]:
y_hats = [model.predict(X_test) for model in model_cat]
acc = np.mean([accuracy_score(y_test , y_hat) for y_hat in y_hats])
print(f'CatB Accuracy(train data) = {acc:.4f}')

CatB Accuracy(train data) = 0.8325


# As you can see, LGBM has the best result for train data

# Prediction for test data and submit the results

In [43]:
lgb = LGBMClassifier(random_state=SEED)
model_lgb = lgb.fit(X, y)

In [44]:
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
predictions = model_lgb.predict(X_df_test)

output = pd.DataFrame({'PassengerId': gender_submission.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
